### Authenticate
Get the client id and secret from env vars for prod.
In dev we can create a new client each time

In [1]:
import { registerSystem } from './helpers/gqlHandlers.ts'
import { authenticate, getTokenForSystemClient } from './helpers/authentication.ts'
import { CLIENT_ID, CLIENT_SECRET } from './helpers/vars.ts'
import { ADMIN_USERNAME, ADMIN_PASSWORD } from './helpers/vars.ts'


let clientId: string | null = null
let clientSecret: string | null = null

if (CLIENT_ID && CLIENT_SECRET) {
  clientId = CLIENT_ID
  clientSecret = CLIENT_SECRET
} else {
  // For dev mode
  const adminToken = await authenticate(ADMIN_USERNAME, ADMIN_PASSWORD)
  const systemRegistration = await registerSystem(adminToken)

  clientId = systemRegistration.data.registerSystem.system.clientId
  clientSecret = systemRegistration.data.registerSystem.clientSecret
}

const sysToken = await getTokenForSystemClient(clientId, clientSecret)

"112a8ed8-f757-4a78-987b-234a23ab4d56"

### Sync locations

On update to v1.9, the postgres db won't have locations so this is required to sync them from mongo

In [2]:
import { syncLocations } from './helpers/gqlHandlers.ts'  

const res = await syncLocations(sysToken)
res

"OK"

### Fetch birth and death events

In [3]:
import { fetchEvents } from './helpers/formsHandlers.ts'
import { extractFieldType } from './helpers/utils.ts'

const events = await fetchEvents(sysToken)

const ignoredFields = ['DIVIDER', 'PARAGRAPH']

const birthEvent = events.find((x) => x.id === 'birth')
const deathEvent = events.find((x) => x.id === 'death')

const birthEventFields = new Set(
  extractFieldType(birthEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)

const deathEventFields = new Set(
  extractFieldType(deathEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)


### Get all potential resolvers
Use only resolvers for used event fields to avoid nulls

In [4]:
import defaultResolvers from './helpers/defaultResolvers.ts'
import { countryResolver } from './countryData/countryResolvers.ts'

const allResolvers = { ...defaultResolvers, ...countryResolver }

const birthResolver = Object.fromEntries(
  Object.entries(allResolvers).filter(([key, _value]) =>
    [...birthEventFields].includes(key)
  )
)

const deathResolver = Object.fromEntries(
  Object.entries(allResolvers).filter(([key, _value]) =>
    [...deathEventFields].includes(key)
  )
)

### Migrate births

In [ ]:
import { EVENT } from './helpers/vars.ts'
import {
  fetchBirthRegistration,
  bulkImport,
  fetchAllBirthRegistrations,
} from './helpers/gqlHandlers.ts'
import { transform } from './helpers/transform.ts'
import { batch } from './helpers/utils.ts'

const saved = []

const migrateBirth = async (entryIds) => {
  const transformed = []

  for (const entryId of entryIds) {
    // Get entry
    const birthRegistrationData = await fetchBirthRegistration(
      entryId,
      sysToken
    )
    if (!birthRegistrationData.data.fetchBirthRegistration) {
      console.error(JSON.stringify(birthRegistrationData, null, 2))
      throw new Error(`No data for ${entryId}`)
    }

    let document
    try {
      document = transform(
        birthRegistrationData.data.fetchBirthRegistration,
        birthResolver
      )
      transformed.push(document)
    } catch (err) {
      // TODO remove for PROD
      console.log('document')
      console.log(JSON.stringify(document, null, 2))
      console.log('birthRegistrationData')
      console.log(JSON.stringify(birthRegistrationData))
      throw err.message
    }
  }


  await bulkImport(transformed, sysToken)
}

const pageSize = 10000
const batchSize = 100
let itemsRemaining = 0
let page = 1

if (EVENT === 'birth') {
  do {
    const birthRegistrations = await fetchAllBirthRegistrations(
      sysToken,
      page,
      pageSize
    )
    if (!birthRegistrations.data.searchEvents) {
      console.error(JSON.stringify(birthRegistrations, null, 2))
      throw new Error('No data from searchEvents')
    }
    
    const birthIds = birthRegistrations.data.searchEvents.results.map(
      (x) => x.id
    )
    console.log('BirthIds found: ', birthIds.length)

    const batches = batch(birthIds, batchSize)

    for (const batch of batches) {
      const savedId = await migrateBirth(batch)
      saved.push(savedId)
    }

    const totalItems = birthRegistrations.data.searchEvents.totalItems
    itemsRemaining = Math.max(0, totalItems - page * pageSize)

    console.log(
      `Processed ${
        page * pageSize - 1 + birthIds.length
      } of ${totalItems} birth registrations... With ${itemsRemaining} remaining.`
    )
    page += 1
  } while (itemsRemaining > 0)
}


BirthIds found:  1
savedId :>>  undefined
Processed 10000 of 1 birth registrations... With 0 remaining.


2

### Migrate Deaths

In [6]:
import {
  fetchDeathRegistration,
  fetchAllDeathRegistrations,
} from './helpers/gqlHandlers.ts'

const migrateDeath = async (entryIds) => {
  const transformed = []

  for (const entryId of entryIds) {
    // Get entry
    const deathRegistrationData = await fetchDeathRegistration(
      entryId,
      sysToken
    )
    if (!deathRegistrationData.data.fetchDeathRegistration) {
      console.error(JSON.stringify(deathRegistrationData, null, 2))
      throw new Error(`No data for ${entryId}`)
    }

    let document
    try {
      document = transform(
        deathRegistrationData.data.fetchDeathRegistration,
        deathResolver
      )
      transformed.push(document)
    } catch (err) {
      // TODO remove for PROD
      console.log('document')
      console.log(JSON.stringify(document, null, 2))
      console.log('deathRegistrationData')
      console.log(JSON.stringify(deathRegistrationData))
      throw err.message
    }
  }

  await bulkImport(transformed, sysToken)
}

const pageSize = 10000
const batchSize = 100
let itemsRemaining = 0
let page = 1

if (EVENT === 'death') {
  do {
    const deathRegistrations = await fetchAllDeathRegistrations(
      sysToken,
      page,
      pageSize
    )
    if (!deathRegistrations.data.searchEvents) {
      console.error(JSON.stringify(deathRegistrations, null, 2))
      throw new Error('No data from searchEvents')
    }
    const deathIds = deathRegistrations.data.searchEvents.results.map(
      (x) => x.id
    )
    console.log('DeathIds found: ', deathIds.length)

    const batches = batch(deathIds, batchSize)

    for (const batch of batches) {
      const savedId = await migrateDeath(batch)
      saved.push(savedId)
    }

    const totalItems = deathRegistrations.data.searchEvents.totalItems
    itemsRemaining = Math.max(0, totalItems - page * pageSize)

    console.log(
      `Processed ${
        page * pageSize - 1 + deathIds.length
      } of ${totalItems} death registrations... With ${itemsRemaining} remaining.`
    )
    page += 1
  } while (itemsRemaining > 0)
}


### Output results


In [7]:
import { DOMAIN } from './helpers/vars.ts'
import { REGISTER_APP } from './helpers/routes.ts'

if (DOMAIN === 'localhost') {
  console.log('🍞 Declarations succesfully migrated:')
  console.log()
  saved.forEach((s) => {
    console.log(
      `You can view the original record [here](${REGISTER_APP}/${s}/viewRecord?V2_EVENTS=false)`
    )
    console.log(
      `and the new migrated record [here](${REGISTER_APP}/events/view/${s}?V2_EVENTS=true)`
    )
  })
} else {
  console.log('Declarations succesfully migrated')
}


🍞 Declarations succesfully migrated:

You can view the original record [here](http://localhost:3000/undefined/viewRecord?V2_EVENTS=false)
and the new migrated record [here](http://localhost:3000/events/view/undefined?V2_EVENTS=true)
